# Executing commands and Playbooks at Nodes and using Role System

This notebook  brief describes how to execute commands in CLAP nodes. There are three ways to execute commands:
* Directly executing shell commands
* Executing an Ansible playbook
* Executing role's actions

In [1]:
import sys
sys.path.append('..')
!pip list

Package                       Version
----------------------------- -----------
alabaster                     0.7.12
ansible                       3.4.0
ansible-base                  2.10.9
ansible-runner                1.4.7
anyio                         3.1.0
argcomplete                   1.12.3
argon2-cffi                   20.1.0
astroid                       2.5.6
async-generator               1.10
attrs                         21.2.0
Babel                         2.9.1
backcall                      0.2.0
bcrypt                        3.2.0
bleach                        3.3.0
boto                          2.49.0
boto3                         1.17.73
botocore                      1.20.73
certifi                       2020.12.5
cffi                          1.14.5
chardet                       4.0.0
click                         8.0.0
coloredlogs                   15.0
contextlib2                   0.6.0.post1
cryptography                  3.4.7
dacite                        1.6.0
d

Let's perform traditional imports

In [2]:
import yaml
from dataclasses import asdict
from app.cli.modules.node import get_config_db, get_node_manager
from app.cli.modules.role import get_role_manager
from clap.utils import float_time_to_string, path_extend
from clap.executor import SSHCommandExecutor, AnsiblePlaybookExecutor

Let's create manager objects

In [8]:
configuration_db = get_config_db()
node_manager = get_node_manager()
role_manager = get_role_manager()
# Private's path (usually ~/.clap/private/) will be used for other methods
private_path = node_manager.private_path

And get nodes in CLAP's system. Here we have 2 nodes, previously created with CLAP

In [6]:
nodes = node_manager.get_all_nodes()
for node in nodes:
    print(f"Node ID: {node.node_id} ({node.nickname}); status: {node.status}; "
          f"IP: {node.ip}; type: {node.configuration.instance.instance_config_id}")

Node ID: 07f4a369663f48d48254f2ad4c5abbfe (HenryRoy); status: reachable; IP: 107.22.143.157; type: type-a
Node ID: 8133c6f7f9ca48258d1e0f01e11326f6 (GeorgeHerndon); status: reachable; IP: 18.207.2.190; type: type-a


Let's update node information using IS alive method

In [7]:
node_ids = [node.node_id for node in nodes]
for node_id, status in node_manager.is_alive(node_ids).items():
    print(f"Node {node_id} is {'alive' if status else 'not alive'}")

Node 07f4a369663f48d48254f2ad4c5abbfe is alive
Node 8133c6f7f9ca48258d1e0f01e11326f6 is alive


## Executing Shell commands directly, using SSHCommandExecutor class

The SSHCommandExecutor class allows you to execute shel commands in nodes. The command is a string of shell commands. This class must be initializated with the command string, the list of **NodeDescriptors** and the path to private directory.

After using the run() method from SSHCommandExecutor, it wil be returned a dataclass called `CommandResult` with the following information:
* ok: if the command was executed in nodes (SSH performed and command executed)
* ret_code: if the command was executed (ok==True), this will contain the return code, otherwise None
* stdout_lines: if the command was executed (ok==True), this will contain a list of strings of the stdout output, otherwise None
* stderr_lines: if the command was executed (ok==True), this will contain a list of strings of the stderr output, otherwise None
* error: if the command was not executed (ok==False), this will contain the string with the exception, otherwise None

In [9]:
command_to_execute = """
git clone https://github.com/lmcad-unicamp/CLAP.git CLAP-ssh
echo Clonned CLAP into CLAP-ssh
"""
executor = SSHCommandExecutor(command_to_execute, nodes, private_path)
result = executor.run()

for node_id, res in result.items():
    print(f"Node id {node_id}, executed the command: {res.ok}, ret code: {res.ret_code}")
    res_dict = asdict(res)
    print('-----')
    print(yaml.dump(res_dict, indent=4, sort_keys=True))

Node id 07f4a369663f48d48254f2ad4c5abbfe, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- 'Cloning into ''CLAP-ssh''...

    '
stdout_lines:
- 'Clonned CLAP into CLAP-ssh

    '

Node id 8133c6f7f9ca48258d1e0f01e11326f6, executed the command: True, ret code: 0
-----
error: null
ok: true
ret_code: 0
stderr_lines:
- 'Cloning into ''CLAP-ssh''...

    '
stdout_lines:
- 'Clonned CLAP into CLAP-ssh

    '



In [ ]:
!cat ~/playbook.yml

In [ ]:
playbook_file = path_extend('~/playbook.yml')
inventory = AnsiblePlaybookExecutor.create_inventory(nodes, private_path)
print(yaml.dump(inventory, indent=4))
executor = AnsiblePlaybookExecutor(playbook_file, private_path, inventory=inventory)
result = executor.run()

In [ ]:
print(f"Did the playbook executed? {result.ok}")
print(f"Ansible playbook return code: {result.ret_code}")
print(f"Let's check how nodes executed: ")
for node_id, status in result.hosts.items():
    print(f"    Node {node_id}: {status}")
print(f"Let's check variables set using set_fact module: ")
for node_id, facts in result.vars.items():
    print(f"    Node {node_id}: {facts}")

# Roles

In [ ]:
!cat ~/.clap/roles/actions.d/commands-common.yaml

In [ ]:
for role_name, role_info in role_manager.roles.items():
    role_dict = asdict(role_info)
    print('------')
    print(f"Role: {role_name}")
    print(f"{yaml.dump(role_dict, indent=4)}")

In [ ]:
added_nodes = role_manager.add_role('commands-common', node_ids)
print(f"Role commands-common was added to {len(added_nodes)} nodes: {node_ids}")

In [ ]:
playbook_result = role_manager.perform_action('commands-common', 'update-packages', node_ids)

In [ ]:
nodes_belonging_to_role = role_manager.get_all_role_nodes('commands-common')
print(nodes_belonging_to_role)

In [ ]:
extra = {'cmd': 'ls -lha'}
result = role_manager.perform_action('commands-common', 'run-command', nodes_belonging_to_role, extra_args=extra)
print(f"Playbok executed correctly: {result.ok}")